<a href="https://colab.research.google.com/github/MinhQuanDangLe/Homemart-Behavior-Analysis-DTSC2024/blob/main/Homemart_Behavior_Analysis_DSTC2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import thư viện, thiết lập đường dẫn vào dataframe

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# Thiết lập đường dẫn đến file CSV
path_customerbehavior_data = "/content/drive/MyDrive/1. PROJECTS/Học Data/01. DSTC 2024 _ TÀI LIỆU BẢNG B/03_Customer_Behavior_Data.csv"
path_item_data = "/content/drive/MyDrive/1. PROJECTS/Học Data/01. DSTC 2024 _ TÀI LIỆU BẢNG B/03_Item_Information_Data.csv"
path_shelf_data = "/content/drive/MyDrive/1. PROJECTS/Học Data/01. DSTC 2024 _ TÀI LIỆU BẢNG B/03_Shelf_Information_Data.csv"
# Đọc file csv vào DataFrame
df = pd.read_csv(path_customerbehavior_data)
df_item = pd.read_csv(path_item_data)
df_shelf =pd.read_csv(path_shelf_data)
# Hiển thị nội dung của DataFrame
print(df)
print(df_item)
print(df_shelf)

# Xử lý missing data ở các cột Putting item into bag, Taking item out of bag, Putting item into bag in the 2nd time



## Xoá các dòng trống cả 3 và điền các dòng có dữ liệu tương tự


In [ ]:
# Cột muốn xử lý missing values
columns_to_check = ["Putting item into bag", "Taking item out of bag", "Putting item into bag in the 2nd time"]

# Cột để nhóm dữ liệu
group_by_columns = ["Shelf ID","Item ID", "Age", "Gender"]

# Loại bỏ các dòng mà cả 3 cột "Putting item into bag", "Taking item out of bag", và "Putting item into bag in the 2nd time" đều bị trống
df = df.dropna(subset=columns_to_check, how='all')

# Hàm để điền giá trị thiếu dựa trên các dòng tương tự
def fill_missing_values(row, df, columns_to_check, group_by_columns):
    # Lấy các giá trị không missing để so sánh
    condition = (df[group_by_columns] == row[group_by_columns]).all(axis=1)
    similar_rows = df[condition & df[columns_to_check].notna().all(axis=1)]

    if len(similar_rows) > 0:
        # Tính giá trị phổ biến nhất cho từng cột
        for col in columns_to_check:
            most_common = similar_rows[col].mode()[0]
            row[col] = most_common

    return row

# Áp dụng hàm trên cho từng dòng
df = df.apply(lambda row: fill_missing_values(row, df, columns_to_check, group_by_columns) if row[columns_to_check].isna().any() else row, axis=1)

In [ ]:
num_empty_rows = df.isnull().any(axis=1).sum()
print (num_empty_rows)

## Kiểm tra nên dùng mô hình nào

In [ ]:
# Tách phần không có missing values
df_no_missing = df.dropna()

# Xác định cột target và features
target_columns = ["Putting item into bag", "Taking item out of bag", "Putting item into bag in the 2nd time"]
features = ["Shelf ID", "Person ID", "Age", "Gender", "Height", "Weight", "Married status",
            "Moving speed (m/s)", "Item ID", "Returning item", "Holding the item (s)",
            "Holding the bag", "Picking up item", "Returning item"]

# Bước 1: Lấy bộ dữ liệu không có missing values (cho các cột mục tiêu)
df_no_missing = df.dropna(subset=target_columns)

# Bước 2: Chuyển đổi các biến phân loại thành số
le = LabelEncoder()
df_no_missing['Gender'] = le.fit_transform(df_no_missing['Gender'])
df_no_missing['Married status'] = le.fit_transform(df_no_missing['Married status'])
df_no_missing['Holding the bag'] = le.fit_transform(df_no_missing['Holding the bag'])
df_no_missing['Picking up item'] = le.fit_transform(df_no_missing['Picking up item'])
df_no_missing['Returning item'] = le.fit_transform(df_no_missing['Returning item'])

# Hàm huấn luyện và đánh giá mô hình
def evaluate_model(X, y, model_name):
    # Chia dữ liệu thành tập huấn luyện và tập kiểm tra
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Kiểm tra và chuyển đổi y_train và y_test nếu cần
    if y_train.dtype != 'int' and y_train.dtype != 'bool':
        y_train = y_train.astype('category').cat.codes
    if y_test.dtype != 'int' and y_test.dtype != 'bool':
        y_test = y_test.astype('category').cat.codes

    # Khởi tạo mô hình dựa trên tên mô hình
    if model_name == "Decision Tree":
        model = DecisionTreeClassifier(random_state=42)
    elif model_name == "Random Forest":
        model = RandomForestClassifier(random_state=42)

    # Huấn luyện mô hình
    model.fit(X_train, y_train)

    # Dự đoán trên tập kiểm tra
    y_pred = model.predict(X_test)

    # Đánh giá mô hình
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=1)
    recall = recall_score(y_test, y_pred, average='weighted')
    conf_matrix = confusion_matrix(y_test, y_pred)

    return accuracy, f1, precision, recall, conf_matrix

# Bước 3: Đánh giá mô hình cho từng cột mục tiêu
results = []

for target_column in target_columns:
    print(f"Evaluating models for target column: {target_column}")

    # Chọn X là các features và y là target column
    X = df_no_missing[features]
    y = df_no_missing[target_column]

    # Ensure the target variable is categorical if it's boolean
    if y.dtype != 'object' and y.dtype != 'bool':
        y = y.astype('category')

    # Huấn luyện và đánh giá Decision Tree
    dt_accuracy, dt_f1, dt_precision, dt_recall, dt_conf_matrix = evaluate_model(X, y, "Decision Tree")
    print(f"Decision Tree - {target_column} - Accuracy: {dt_accuracy}, F1 Score: {dt_f1}, Precision: {dt_precision}, Recall: {dt_recall}")
    print(f"Confusion Matrix:\n{dt_conf_matrix}")

    # Huấn luyện và đánh giá Random Forest
    rf_accuracy, rf_f1, rf_precision, rf_recall, rf_conf_matrix = evaluate_model(X, y, "Random Forest")
    print(f"Random Forest - {target_column} - Accuracy: {rf_accuracy}, F1 Score: {rf_f1}, Precision: {rf_precision}, Recall: {rf_recall}")
    print(f"Confusion Matrix:\n{rf_conf_matrix}")

    # Lưu kết quả cho Decision Tree và Random Forest
    results.append({
        "Target Column": target_column,
        "Model": "Decision Tree",
        "Accuracy": dt_accuracy,
        "F1 Score": dt_f1,
        "Precision": dt_precision,
        "Recall": dt_recall,
        "Confusion Matrix": dt_conf_matrix
    })
    results.append({
        "Target Column": target_column,
        "Model": "Random Forest",
        "Accuracy": rf_accuracy,
        "F1 Score": rf_f1,
        "Precision": rf_precision,
        "Recall": rf_recall,
        "Confusion Matrix": rf_conf_matrix
    })

# Chuyển kết quả thành DataFrame (loại bỏ confusion matrix)
results_df = pd.DataFrame([{key: value for key, value in result.items() if key != 'Confusion Matrix'} for result in results])

# Hiển thị kết quả
print(results_df)

# Lưu kết quả ra file CSV nếu cần
results_df.to_csv("/content/drive/MyDrive/1. PROJECTS/Học Data/01. DSTC 2024 _ TÀI LIỆU BẢNG B/model_comparison_results.csv", index=False)


In [ ]:
# Hàm tạo và hiển thị ma trận nhầm lẫn
def plot_confusion_matrix(cm, classes, title='Confusion Matrix', cmap=plt.cm.Blues):
    """
    Tạo và hiển thị ma trận nhầm lẫn.
    :param cm: Ma trận nhầm lẫn
    :param classes: Danh sách các lớp
    :param title: Tiêu đề của biểu đồ
    :param cmap: Bảng màu
    """
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap=cmap,
                xticklabels=classes, yticklabels=classes)
    plt.xlabel('Predicted labels')
    plt.ylabel('True labels')
    plt.title(title)
    plt.show()

# Sử dụng hàm plot_confusion_matrix
target_columns = ["Putting item into bag", "Taking item out of bag", "Putting item into bag in the 2nd time"]
classes = ['False', 'True']  # Cập nhật nếu lớp của bạn khác

for target_column in target_columns:
    print(f"Confusion Matrix for {target_column}")

    # Chọn X là các features và y là target column
    X = df_no_missing[features]
    y = df_no_missing[target_column]

    # Đánh giá Decision Tree
    dt_accuracy, dt_f1, dt_precision, dt_recall, dt_conf_matrix = evaluate_model(X, y, "Decision Tree")
    print(f"Decision Tree - {target_column} - Accuracy: {dt_accuracy}, F1 Score: {dt_f1}, Precision: {dt_precision}, Recall: {dt_recall}")
    print(f"Confusion Matrix:\n{dt_conf_matrix}")

    # Trực quan hóa ma trận nhầm lẫn của Decision Tree
    plot_confusion_matrix(dt_conf_matrix, classes, title=f'Decision Tree Confusion Matrix for {target_column}')

    # Đánh giá Random Forest
    rf_accuracy, rf_f1, rf_precision, rf_recall, rf_conf_matrix = evaluate_model(X, y, "Random Forest")
    print(f"Random Forest - {target_column} - Accuracy: {rf_accuracy}, F1 Score: {rf_f1}, Precision: {rf_precision}, Recall: {rf_recall}")
    print(f"Confusion Matrix:\n{rf_conf_matrix}")

    # Trực quan hóa ma trận nhầm lẫn của Random Forest
    plot_confusion_matrix(rf_conf_matrix, classes, title=f'Random Forest Confusion Matrix for {target_column}')

### Nhận xét
1. Putting item into bag:
  Random Forest có hiệu suất tốt hơn đáng kể so với Decision Tree. Đặc biệt là ở chỉ số Accuracy và Recall, điều này cho thấy Random Forest có khả năng phân loại chính xác hơn và ít bỏ sót hơn so với Decision Tree.

2. Taking item out of bag:
  Tương tự, Random Forest cho kết quả tốt hơn ở tất cả các chỉ số. Sự chênh lệch không quá lớn nhưng vẫn cho thấy Random Forest có hiệu suất phân loại tốt hơn một chút.

3. Putting item into bag in the 2nd time:
  Mặc dù cả hai mô hình đều có hiệu suất không cao đối với target này, Random Forest vẫn cho kết quả tốt hơn ở tất cả các chỉ số so với Decision Tree. Điều này nhấn mạnh tính hiệu quả của Random Forest trong việc xử lý nhiều lần thao tác lặp lại.
4. Trong cả ba trường hợp, Random Forest đều cho kết quả tốt hơn so với Decision Tree trên tất cả các chỉ số (Accuracy, F1 Score, Precision, và Recall). Đặc biệt, Random Forest có khả năng xử lý tốt hơn trong việc tránh overfitting và cải thiện độ chính xác tổng thể. Sự khác biệt rõ rệt nhất là ở target "Putting item into bag", với Random Forest cho độ chính xác và độ nhạy cao hơn đáng kể.







## Dùng mô hình Decision Tree để điền các trường hợp còn trống

In [ ]:
# Tạo bản sao của df để lưu kết quả của Decision Tree
df_dt = df.copy()

# Danh sách các cột cần dự đoán
columns_to_predict = ["Putting item into bag", "Taking item out of bag", "Putting item into bag in the 2nd time"]

# Các cột đặc trưng để huấn luyện mô hình (sử dụng các thông tin không bị thiếu)
features = ["Shelf ID", "Person ID", "Age", "Gender", "Height", "Weight", "Married status",
            "Moving speed (m/s)", "Item ID", "Returning item", "Holding the item (s)",
            "Holding the bag", "Picking up item", "Returning item"]

# Bước 1: Chuẩn bị dữ liệu
# Chuyển các biến phân loại thành số (Label Encoding)
le = LabelEncoder()
df_dt["Gender"] = le.fit_transform(df_dt["Gender"])
df_dt["Married status"] = le.fit_transform(df_dt["Married status"])
df_dt["Holding the bag"] = le.fit_transform(df_dt["Holding the bag"])
df_dt["Picking up item"] = le.fit_transform(df_dt["Picking up item"])
df_dt["Returning item"] = le.fit_transform(df_dt["Returning item"])

# Hàm xử lý để huấn luyện và dự đoán cho từng cột
def predict_missing_values_with_dt(df, target_column, features):
    # Tách dữ liệu thành 2 phần:
    # - train_data: Dữ liệu có sẵn, không thiếu giá trị
    # - test_data: Dữ liệu thiếu (NaN) cần dự đoán
    train_data = df.dropna(subset=[target_column])
    test_data = df[df[target_column].isna()]

    # X tách ra các biến đặc trưng (features), y là cột mục tiêu (target)
    X_train = train_data[features]
    y_train = train_data[target_column]
    X_test = test_data[features]

    # Kiểm tra kiểu dữ liệu của cột mục tiêu và chuyển đổi nếu cần
    if y_train.dtype not in ['int64', 'int32', 'int16', 'category']:
        y_train = y_train.astype('category').cat.codes

    # Huấn luyện mô hình Decision Tree
    model = DecisionTreeClassifier()
    model.fit(X_train, y_train)

    # Dự đoán giá trị cho phần test_data (bị thiếu)
    y_pred = model.predict(X_test)

    # Điền giá trị đã dự đoán vào cột bị thiếu
    df.loc[df[target_column].isna(), target_column] = y_pred

      # Chuyển đổi giá trị từ 0/1 thành FALSE/TRUE
    df[target_column] = df[target_column].replace({0: 'FALSE', 1: 'TRUE'})

    # Trả về DataFrame đã được điền đầy đủ
    return df

# Bước 2: Dự đoán từng cột bị thiếu và lưu kết quả vào df_dt
for col in columns_to_predict:
    df_dt = predict_missing_values_with_dt(df_dt, col, features)

# df_dt bây giờ đã chứa dữ liệu với các giá trị bị thiếu được điền bởi Decision Tree
print(df_dt)


       Shelf ID  Person ID   Timestamp  Age  Gender  Height  Weight  \
0             4          1  1720260000   52       0     162      41   
1             4          1  1720260120   52       0     162      41   
2             4          1  1720260300   52       0     162      41   
3             4          1  1720260360   52       0     162      41   
4             4          1  1720260480   52       0     162      41   
...         ...        ...         ...  ...     ...     ...     ...   
15389         2       1500  1720151340   52       0     154      44   
15390         3       1500  1720151640   52       0     154      44   
15391         3       1500  1720151760   52       0     154      44   
15392         3       1500  1720151880   52       0     154      44   
15393         3       1500  1720152180   52       0     154      44   

       Married status  Moving speed (m/s)  Item ID  Looking at item (s)  \
0                   0                0.19       16                   27 

## Dùng mô hình Random Forest để điền các trường hợp còn trống

In [ ]:
# Tạo bản sao của df để lưu kết quả của Random Forest
df_rf = df.copy()

# Danh sách các cột cần dự đoán
columns_to_predict = ["Putting item into bag", "Taking item out of bag", "Putting item into bag in the 2nd time"]

# Các cột feature
features = ["Shelf ID", "Person ID", "Age", "Gender", "Height", "Weight", "Married status",
            "Moving speed (m/s)", "Item ID", "Returning item", "Holding the item (s)",
            "Holding the bag", "Picking up item", "Returning item"]

# Bước 1: Chuẩn bị dữ liệu
# Chuyển các biến phân loại thành số (Label Encoding)
le = LabelEncoder()
df_rf["Gender"] = le.fit_transform(df_rf["Gender"])
df_rf["Married status"] = le.fit_transform(df_rf["Married status"])
df_rf["Holding the bag"] = le.fit_transform(df_rf["Holding the bag"])
df_rf["Picking up item"] = le.fit_transform(df_rf["Picking up item"])
df_rf["Returning item"] = le.fit_transform(df_rf["Returning item"])

# Hàm xử lý để huấn luyện và dự đoán cho từng cột
def predict_missing_values(df, target_column, features):
    # Tách dữ liệu thành 2 phần:
    # - train_data: Dữ liệu có sẵn, không thiếu giá trị
    # - test_data: Dữ liệu thiếu (NaN) cần dự đoán
    train_data = df.dropna(subset=[target_column])
    test_data = df[df[target_column].isna()]

    # X tách ra các biến đặc trưng (features), y là cột mục tiêu (target)
    X_train = train_data[features]
    y_train = train_data[target_column]
    X_test = test_data[features]

    # Kiểm tra kiểu dữ liệu của cột mục tiêu và chuyển đổi nếu cần
    if y_train.dtype not in ['int64', 'int32', 'int16', 'category']:
        y_train = y_train.astype('category').cat.codes

    # Huấn luyện mô hình Random Forest
    model = RandomForestClassifier()
    model.fit(X_train, y_train)

    # Dự đoán giá trị cho phần test_data (bị thiếu)
    y_pred = model.predict(X_test)

    # Điền giá trị đã dự đoán vào cột bị thiếu
    df.loc[df[target_column].isna(), target_column] = y_pred

     # Chuyển đổi giá trị từ 0/1 thành FALSE/TRUE
   # df[target_column] = df[target_column].replace({0: 'FALSE', 1: 'TRUE'})

    # Trả về DataFrame đã được điền đầy đủ
    return df

# Bước 2: Dự đoán từng cột bị thiếu của df_rf
for col in columns_to_predict:
    df_rf = predict_missing_values(df_rf, col, features)

# Bây giờ df_rf đã được điền đầy đủ các giá trị bị thiếu cho các cột dự đoán
print(df_rf)



       Shelf ID  Person ID   Timestamp  Age  Gender  Height  Weight  \
0             4          1  1720260000   52       0     162      41   
1             4          1  1720260120   52       0     162      41   
2             4          1  1720260300   52       0     162      41   
3             4          1  1720260360   52       0     162      41   
4             4          1  1720260480   52       0     162      41   
...         ...        ...         ...  ...     ...     ...     ...   
15389         2       1500  1720151340   52       0     154      44   
15390         3       1500  1720151640   52       0     154      44   
15391         3       1500  1720151760   52       0     154      44   
15392         3       1500  1720151880   52       0     154      44   
15393         3       1500  1720152180   52       0     154      44   

       Married status  Moving speed (m/s)  Item ID  Looking at item (s)  \
0                   0                0.19       16                   27 

In [ ]:
df_rf.isnull().sum()

,0
Shelf ID,0
Person ID,0
Timestamp,0
Age,0
Gender,0
Height,0
Weight,0
Married status,0
Moving speed (m/s),0
Item ID,0


## So sánh 2 dataframe của 2 phương pháp

In [ ]:
df_rf_nullcheck = df_rf.isnull().any(axis=1).sum()
print (df_rf_nullcheck)

df_dt_nullcheck = df_dt.isnull().any(axis=1).sum()
print (df_dt_nullcheck)

### In ra các dòng có giá trị khác biệt bằng .compare

In [ ]:
# So sánh các giá trị không trùng lặp giữa df và df_dt
comparison_df = df_rf.compare(df_dt)

# In ra những dòng có giá trị khác biệt
print(comparison_df)

comparison_df.to_csv("/content/drive/MyDrive/1. PROJECTS/Học Data/01. DSTC 2024 _ TÀI LIỆU BẢNG B/differences_between_df_and_df_dt.csv", index=False)


### In ra các dòng điền missing value của Dataframe Random Forest khác với Decision Tree bằng .ne

In [ ]:
# So sánh df_rf và df_dt
difference_mask = df_rf.ne(df_dt)

# Lọc ra những dòng từ df_rf có giá trị khác  so với df_dt
rows_with_differences = df_rf[difference_mask.any(axis=1)]

# In ra DataFrame chứa các dòng của df_rf có giá trị khác  so với df_dt
print(rows_with_differences)

# Lưu kết quả vào file CSV
rows_with_differences.to_csv("/content/drive/MyDrive/1. PROJECTS/Học Data/01. DSTC 2024 _ TÀI LIỆU BẢNG B/rows_with_differences_between_df_rf_and_df_dt.csv", index=False)


# Trim dataframe

## Thực hiện Trim dataframe

In [ ]:
# Hàm trim khoảng trắng ở hai bên cho tất cả các cột và dòng trong DataFrame
def trim_all_columns(df):
    # Áp dụng trim cho tiêu đề cột
    df.columns = df.columns.str.strip()
    # Áp dụng trim cho tất cả các cột có kiểu chuỗi (string) trong dữ liệu
    return df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# Trim cho cả 3 DataFrame
df_rf = trim_all_columns(df_rf)
df_item = trim_all_columns(df_item)
df_shelf = trim_all_columns(df_shelf)

print(df_rf.columns)
print(df_item.columns)
print(df_shelf.columns)

Index(['Shelf ID', 'Person ID', 'Timestamp', 'Age', 'Gender', 'Height',
       'Weight', 'Married status', 'Moving speed (m/s)', 'Item ID',
       'Looking at item (s)', 'Holding the item (s)', 'Holding the bag',
       'Picking up item', 'Returning item', 'Putting item into bag',
       'Taking item out of bag', 'Putting item into bag in the 2nd time'],
      dtype='object')
Index(['Shelf ID', 'Item ID', 'Name', 'Origin', 'Location', 'Weight (g)',
       'Price', 'NSX', 'HSD', 'Marketing strategy', 'Discount', 'Rate'],
      dtype='object')
Index(['Shelf ID', 'Description', 'Location_x (center)', 'Location_y (center)',
       'Width', 'Height', 'Length', 'Number of items', 'Shape'],
      dtype='object')


<ipython-input-6-5ef6a70e59dd>:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
<ipython-input-6-5ef6a70e59dd>:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
<ipython-input-6-5ef6a70e59dd>:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


# Xử lý kiểu dữ liệu

In [ ]:
print(df_rf.dtypes)
print(df_item.dtypes)
print(df_shelf.dtypes)

Shelf ID                                   int64
Person ID                                  int64
Timestamp                                  int64
Age                                        int64
Gender                                     int64
Height                                     int64
Weight                                     int64
Married status                             int64
Moving speed (m/s)                       float64
Item ID                                    int64
Looking at item (s)                        int64
Holding the item (s)                       int64
Holding the bag                            int64
Picking up item                            int64
Returning item                             int64
Putting item into bag                     object
Taking item out of bag                    object
Putting item into bag in the 2nd time     object
Buy Checking                              object
dtype: object
Shelf ID                  int64
Item ID                

## Kiểm tra kiểu dữ liệu các cột ở 3 data frame

## Đổi kiểu dữ liệu cột Price sang Int

In [ ]:
df_rf.dtypes
df_item.dtypes
df_shelf.dtypes

,0
Shelf ID,int64
Description,object
Location_x (center),int64
Location_y (center),int64
Width,int64
Height,int64
Length,int64
Number of items,int64
Shape,object


In [ ]:
# Loại bỏ dấu phân cách hàng nghìn (dấu phẩy) trong cột 'Price'
df_item['Price'] = df_item['Price'].str.replace(',', '')

# Chuyển cột 'Price' sang kiểu số (có thể là int hoặc float, tùy thuộc vào dữ liệu)
df_item['Price'] = pd.to_numeric(df_item['Price'])

# Thêm các cột cần thiết

## Thêm cột kiểm tra mua hàng vào df_rf

In [ ]:
# Điều kiện để kiểm tra xem mặt hàng có được mua hay không
def buy_checking(row):
    if row['Putting item into bag'] and row['Taking item out of bag'] and not row['Putting item into bag in the 2nd time']:
        return "Không mua"
    elif not row['Putting item into bag'] and row['Taking item out of bag'] and not row['Putting item into bag in the 2nd time']:
        return "Không mua"
    elif not row['Putting item into bag'] and not row['Taking item out of bag'] and not row['Putting item into bag in the 2nd time']:
        return "Không mua"
    else:
        return "Mua"

# Áp dụng hàm cho dataframe và tạo cột mới 'Buy Checking'
df_rf['Buy Checking'] = df_rf.apply(buy_checking, axis=1)

# Hiển thị 5 hàng đầu tiên để kiểm tra kết quả
df_rf[['Putting item into bag', 'Taking item out of bag', 'Putting item into bag in the 2nd time', 'Buy Checking']].head()

,Putting item into bag,Taking item out of bag,Putting item into bag in the 2nd time,Buy Checking
0,False,False,True,Mua
1,True,True,True,Mua
2,True,True,False,Không mua
3,False,False,False,Không mua
4,False,False,True,Mua


## Thêm cột Discount checking vào df_item

In [ ]:
# Sử dụng np.where
df_item['Discount checking'] = np.where(df_item['Discount'] > 0, "Có giảm giá", "Không giảm giá")
# Sử dụng .apply
#df_item['Discount checking'] = df_item['Discount'].apply(lambda discount: "Có giảm giá" if discount > 0 else "Không giảm giá")


## Thêm cột giá sau giảm giá vào df_item

In [ ]:
df_item['Price'] = df_item['Price'].astype('float')

# Tạo cột "Price After Discount" trong dataframe df_item
df_item['Price After Discount'] = df_item['Price'] * (1 - df_item['Discount'].astype(float) / 100)

## Thêm cột Revenue vào df_rf


### Cách 2: Sử dụng dictionary

In [ ]:
# Tạo một dictionary từ df_item để tham chiếu Price After Discount theo ItemID và Shelf ID
price_dict = df_item.set_index(['Shelf ID', 'Item ID'])['Price After Discount'].to_dict() # Thay ItemID thành 'Item ID'

# Thêm cột Revenue vào df_rf dựa trên việc kiểm tra điều kiện "Mua"
df_rf['Revenue'] = df_rf.apply(lambda row: price_dict.get((row['Shelf ID'], row['Item ID']), 0) if row['Buy Checking'] == 'Mua' else 0, axis=1) # Thay ItemID thành 'Item ID'

# Hiển thị 5 hàng đầu tiên để kiểm tra kết quả
print(df_rf[['Item ID', 'Shelf ID', 'Buy Checking', 'Revenue']].head())

## Thêm cột Shelf Item ID ở Item ID (optional)

In [ ]:
# Tạo cột mới Shelf Item ID bằng cách kết hợp cột Shelf ID và Item ID
df_rf['Shelf Item ID'] = df_rf['Shelf ID'].astype(str) + '-' + df_rf['Item ID'].astype(str)
df_item['Shelf Item ID'] = df_item['Shelf ID'].astype(str) + '-' + df_item['Item ID'].astype(str)
df_rf.head()
df_item.head()

,Shelf ID,Item ID,Name,Origin,Location,Weight (g),Price,NSX,HSD,Marketing strategy,Discount,Rate,Discount checking,Price After Discount,Shelf Item ID
0,0,0,Bánh Oreo,Vietnam,Cao,120.0,17500.0,14/11/2023,17/10/2024,False,10,4,Có giảm giá,15750.0,0-0
1,0,1,Bánh Chocopie,Vietnam,Cao,396.0,48000.0,07/10/2023,16/09/2024,False,0,3,Không giảm giá,48000.0,0-1
2,0,2,Kẹo cao su Cool Air Fresh,Vietnam,Trung bình,40.0,40000.0,02/06/2024,02/06/2026,False,0,3,Không giảm giá,40000.0,0-2
3,0,3,Bánh mì chà bông Kinh Đô,Vietnam,Trung bình,80.0,13400.0,02/05/2024,02/05/2026,False,0,3,Không giảm giá,13400.0,0-3
4,0,4,Bánh trứng Custard,ThÃ¡i Lan,Trung bình,46.0,17000.0,30/10/2023,16/04/2024,True,10,4,Có giảm giá,15300.0,0-4


# Sắp xếp mặt hàng

## Tỉ lệ picking up item

In [ ]:
# Tính tổng số lần tương tác theo từng cặp Shelf ID và Item ID lưu vào dataframe
total_interactions = df_rf.groupby(['Shelf ID', 'Item ID']).size().reset_index(name='Interaction Count')


In [ ]:
# Đếm số lần khách hàng có hành động "picking up item" (True) theo từng cặp Shelf ID và Item ID
picking_up = df_rf[df_rf['Picking up item'] == True].groupby(['Shelf ID', 'Item ID']).size().reset_index(name='Pick Up Count')

# Merge với df_item để thêm tên sản phẩm (Item Name) từ df_item dựa trên Shelf ID và Item ID
picking_up = picking_up.merge(df_item[['Shelf ID', 'Item ID', 'Name']], left_on=['Shelf ID', 'Item ID'], right_on=['Shelf ID', 'Item ID'], how='left')



# Merge total_interactions with picking_up
# picking_up = picking_up.merge(total_interactions, left_on=['Shelf ID', 'Item ID'], right_on=['Shelf ID', 'Item ID'], how='left')

# Tính tỷ lệ Pick Up Rate
picking_up['Pick Up Rate'] = picking_up['Pick Up Count'] / total_interactions['Interaction Count']

picking_up.head()

,Shelf ID,Item ID,Pick Up Count,Name,Pick Up Rate
0,0,0,205,Bánh Oreo,1.0
1,0,1,220,Bánh Chocopie,1.0
2,0,2,211,Kẹo cao su Cool Air Fresh,1.0
3,0,3,200,Bánh mì chà bông Kinh Đô,1.0
4,0,4,230,Bánh trứng Custard,1.0


In [ ]:
# Lọc các sản phẩm có tỷ lệ Pick Up Rate < 100%
low_pickup_rate_df = picking_up[picking_up['Pick Up Rate'] < 1]
low_pickup_rate_df
# Hiển thị các sản phẩm và kệ hàng có tỷ lệ Pick Up Rate < 100%
#low_pickup_rate_df = low_pickup_rate_df[['Name', 'Shelf ID', 'Pick Up Rate']]

,Shelf ID,Item ID,Pick Up Count,Name,Pick Up Rate
100,6,0,63,Gấu bông Miyu,0.984375
101,6,1,77,Siêu nhân Marvel,0.974684
102,6,2,53,Sách cho trẻ sơ sinh đến 6 tháng tuổi,0.946429
103,6,3,55,Sách cho trẻ từ 1-2 tuổi,0.982143
106,6,6,52,Sách Chào con Ba mẹ đã sẵn sàng,0.981132
107,6,7,59,Sách - Nuôi Con không phải là cuộc chiến Easy,0.967213
108,6,8,64,Sách Cẩm Nang Mang Thai & Sinh con,0.984615
109,6,9,60,Truyện tranh Conan,0.983607
110,6,10,53,Truyện Doraemon,0.963636
113,6,13,57,Sách Thép Đã Tôi Thế Đấy,0.966102


Tỉ lệ pick up rate hầu hết đều là 100% ở tất cả các sản phẩm, trừ một số mặt hàng ở **kệ 6** tỉ lệ này nằm khoảng từ 96.3-98.4%

## Looking và Holding Item

In [ ]:
look_and_hold_time.head()

,Shelf ID,Item ID,Looking at item (s),Holding the item (s),Interaction Count,Name
0,0,0,5518,5353,205,Bánh Oreo
1,0,1,5873,5987,220,Bánh Chocopie
2,0,2,5598,5651,211,Kẹo cao su Cool Air Fresh
3,0,3,5388,5484,200,Bánh mì chà bông Kinh Đô
4,0,4,6145,6230,230,Bánh trứng Custard


In [ ]:
# Tính tổng thời gian Looking và Holding cho từng sản phẩm
look_and_hold_time = df_rf.groupby(['Shelf ID', 'Item ID'])[['Looking at item (s)', 'Holding the item (s)']].sum().reset_index()

# Merge time_sum và total_interactions để có đầy đủ thông tin
look_and_hold_time = look_and_hold_time.merge(total_interactions, on=['Shelf ID', 'Item ID'], how='left')

look_and_hold_time = look_and_hold_time.merge(df_item[['Shelf ID', 'Item ID', 'Name']], left_on=['Shelf ID', 'Item ID'], right_on=['Shelf ID', 'Item ID'], how='left')

# Tính trung bình thời gian cho từng sản phẩm
look_and_hold_time['Average Time (Looking + Holding)'] = (look_and_hold_time['Looking at item (s)'] + look_and_hold_time['Holding the item (s)']) / look_and_hold_time['Interaction Count']
# Calculate average looking time
look_and_hold_time['Average Time (Looking)'] = look_and_hold_time['Looking at item (s)'] / look_and_hold_time['Interaction Count']

# Calculate avlook_and_hold_timeerage holding time
look_and_hold_time['Average Time (Holding)'] = look_and_hold_time['Holding the item (s)'] / look_and_hold_time['Interaction Count']

# Display the result
look_and_hold_time[['Shelf ID','Item ID','Average Time (Looking)', 'Average Time (Holding)','Average Time (Looking + Holding)']].head()

,Shelf ID,Item ID,Average Time (Looking),Average Time (Holding),Average Time (Looking + Holding)
0,0,0,26.917073,26.112195,53.029268
1,0,1,26.695455,27.213636,53.909091
2,0,2,26.530806,26.781991,53.312796
3,0,3,26.940000,27.420000,54.360000
4,0,4,26.717391,27.086957,53.804348


In [ ]:
# Sắp xếp theo cột 'Average Time (Looking + Holding)' theo thứ tự tăng dần
look_and_hold_time_sorted = look_and_hold_time.sort_values(by='Average Time (Looking + Holding)', ascending=True)
look_and_hold_time_sorted[['Shelf ID','Item ID','Average Time (Looking)', 'Average Time (Holding)','Average Time (Looking + Holding)']].head()

,Shelf ID,Item ID,Average Time (Looking),Average Time (Holding),Average Time (Looking + Holding)
27,1,14,6.571429,6.674603,13.246032
26,1,13,7.037879,6.954545,13.992424
110,6,10,12.490909,12.563636,25.054545
103,6,3,12.392857,12.678571,25.071429
114,6,14,13.716981,13.886792,27.603774


In [ ]:
# Tính tứ phân vị cho 'Average Time (Looking)', 'Average Time (Holding)', và 'Average Time (Looking + Holding)'
quartiles_looking = look_and_hold_time['Average Time (Looking)'].quantile([0.25, 0.5, 0.75])
quartiles_holding = look_and_hold_time['Average Time (Holding)'].quantile([0.25, 0.5, 0.75])
quartiles_total = look_and_hold_time['Average Time (Looking + Holding)'].quantile([0.25, 0.5, 0.75])

# Hiển thị tứ phân vị
print("Tứ phân vị Average Time (Looking):")
print(quartiles_looking)
print("\nTứ phân vị Average Time (Holding):")
print(quartiles_holding)
print("\nTứ phân vị Average Time (Looking + Holding):")
print(quartiles_total)


# Hàm phân loại theo tứ phân vị cho từng cột
def classify_quartile(value, quartiles):
    if value <= quartiles[0.25]:
        return 'Q1'
    elif value <= quartiles[0.50]:
        return 'Q2'
    elif value <= quartiles[0.75]:
        return 'Q3'
    else:
        return 'Q4'

# Áp dụng hàm phân loại cho từng cột
look_and_hold_time['Quartile (Looking)'] = look_and_hold_time['Average Time (Looking)'].apply(classify_quartile, args=(quartiles_looking,))
look_and_hold_time['Quartile (Holding)'] = look_and_hold_time['Average Time (Holding)'].apply(classify_quartile, args=(quartiles_holding,))
look_and_hold_time['Quartile (Looking + Holding)'] = look_and_hold_time['Average Time (Looking + Holding)'].apply(classify_quartile, args=(quartiles_total,))


# Sắp xếp các mặt hàng thuộc từng tứ phân vị cho 'Average Time (Looking)', 'Average Time (Holding)', và 'Average Time (Looking + Holding)'
for quartile in ['Q1', 'Q2', 'Q3', 'Q4']:
    print(f"\nMặt hàng thuộc {quartile} (Looking) - Sắp xếp từ nhỏ đến lớn:")
    print(look_and_hold_time[look_and_hold_time['Quartile (Looking)'] == quartile][['Shelf ID', 'Item ID', 'Average Time (Looking)', 'Quartile (Looking)']].sort_values(by='Average Time (Looking)'))

    print(f"\nMặt hàng thuộc {quartile} (Holding) - Sắp xếp từ nhỏ đến lớn:")
    print(look_and_hold_time[look_and_hold_time['Quartile (Holding)'] == quartile][['Shelf ID', 'Item ID', 'Average Time (Holding)', 'Quartile (Holding)']].sort_values(by='Average Time (Holding)'))

    print(f"\nMặt hàng thuộc {quartile} (Looking + Holding) - Sắp xếp từ nhỏ đến lớn:")
    print(look_and_hold_time[look_and_hold_time['Quartile (Looking + Holding)'] == quartile][['Shelf ID', 'Item ID', 'Average Time (Looking + Holding)', 'Quartile (Looking + Holding)']].sort_values(by='Average Time (Looking + Holding)'))


In [ ]:
# Tạo mới một Google Sheet
spreadsheet = gc.create('Look and Hold Time Quartiles')

# Chọn Sheet đầu tiên
worksheet = spreadsheet.get_worksheet(0)

# Chuyển đổi DataFrame thành danh sách 2D để xuất vào Google Sheets
worksheet.update([look_and_hold_time.columns.values.tolist()] + look_and_hold_time.values.tolist())

spreadsheet_url = f'https://docs.google.com/spreadsheets/d/{spreadsheet.id}'
print(f"Google Sheet URL: {spreadsheet_url}")


Google Sheet URL: https://docs.google.com/spreadsheets/d/1LU7IomHdCLgmgYudUb-ivZIHVfK4XHh18jucuNJCBrs


Kệ 6 có đa số các mặt hàng có  'Average Time (Looking)', 'Average Time (Holding) nằm ở tứ phân vị thứ nhất

In [ ]:
# Lọc ra các mặt hàng thuộc kệ 6
shelf_6_items = look_and_hold_time[look_and_hold_time['Shelf ID'] == 6]

# Chọn các cột cần hiển thị
shelf_6_items = shelf_6_items[['Shelf ID', 'Item ID', 'Average Time (Looking)',
                                        'Average Time (Holding)', 'Average Time (Looking + Holding)',
                                        'Quartile (Looking)', 'Quartile (Holding)', 'Quartile (Looking + Holding)']]
shelf_6_items

Filter các mặt hàng kệ 6, có thể thấy trừ mặt hàng 6-11 và 6-12 là sách Nhà Giả Kim và sách Dảk Nhân Tâm là có 'Average Time (Looking)', 'Average Time (Holding) cao và nằm trong tứ phân vị 4 => Được quân tâm nhiều. Còn các mặt hàng khác của kệ 6 hầu hết đều nằm ở Q1 => Được quan tâm ít. Có thể cân nhắc điều chỉnh vị trí của các mặt hàng ở cột 6 ở nơi thu hút hơn.

In [ ]:
# Lọc ra các mặt hàng thuộc kệ 6 từ df_item
shelf_6_items = df_item[df_item['Shelf ID'] == 6]

# Chọn các cột cần hiển thị từ df_item
shelf_6_columns = shelf_6_items[['Name', 'Origin', 'Location', 'Weight (g)', 'Price', 'NSX', 'HSD', 'Marketing strategy', 'Discount', 'Rate','Price After Discount']]
shelf_6_columns

## Tỷ lệ mua

In [ ]:
# Tính tổng số lần tương tác theo từng cặp Shelf ID và Item ID lưu vào dataframe
total_interactions = df_rf.groupby(['Shelf ID', 'Item ID']).size().reset_index(name='Interaction Count')

# Tính số lượng mua cho mỗi mặt hàng
buy_count = df_rf[df_rf['Buy Checking'] == 'Mua'].groupby(['Shelf ID','Item ID']).size().reset_index(name='Buy Count')

# Tính tổng số lần xem cho mỗi mặt hàng

# Tính tỷ lệ mua cho mỗi mặt hàng
buy_ratio = buy_count['Buy Count'] / total_interactions['Interaction Count']*100

# Thêm cột tỷ lệ mua vào DataFrame
df_item['Purchase Ratio'] = buy_ratio



# Xuất dữ liệu sau khi xử lý

In [ ]:
# Xuất dữ liệu sau khi xử lý
df.to_csv("/content/drive/MyDrive/1. PROJECTS/Học Data/01. DSTC 2024 _ TÀI LIỆU BẢNG B/03_Customer_Behavior_Data_After handdling missing.csv", index=False)
df_dt.to_csv("/content/drive/MyDrive/1. PROJECTS/Học Data/01. DSTC 2024 _ TÀI LIỆU BẢNG B/03_Customer_Behavior_Data_Decision Tree.csv", index=False)
df_rf.to_csv("/content/drive/MyDrive/1. PROJECTS/Học Data/01. DSTC 2024 _ TÀI LIỆU BẢNG B/03_Customer_Behavior_Data_Random Forest.csv", index=False)

In [ ]:
df_rf.to_csv("/content/drive/MyDrive/1. PROJECTS/Học Data/01. DSTC 2024 _ TÀI LIỆU BẢNG B/03_Customer_Behavior_Data_Random Forest.csv", index=False)